# Como Usar Variáveis de Ambiente (.env) em Notebooks

Este notebook demonstra como carregar e usar variáveis de ambiente de um arquivo `.env` no seu projeto MLOps.

## 1. Instalar python-dotenv

Primeiro, certifique-se de ter o pacote `python-dotenv` instalado:

```bash
conda install -c conda-forge python-dotenv
```

Ou adicione ao `requirements_conda.txt`:
```
python-dotenv
```

In [ ]:
# Importar bibliotecas necessárias
from dotenv import load_dotenv
import os

## 2. Carregar o arquivo .env

O arquivo `.env` deve estar na raiz do projeto. Você pode criar um copiando o `.env.example`:

```bash
cp .env.example .env
```

**Importante:** O arquivo `.env` não deve ser versionado no Git (já está no `.gitignore`).

In [ ]:
# Carregar variáveis do arquivo .env
load_dotenv()

print("✅ Variáveis de ambiente carregadas do arquivo .env")

## 3. Acessar variáveis de ambiente

Use `os.getenv()` ou `os.environ.get()` para acessar as variáveis:

In [ ]:
# Acessar variáveis de ambiente
mlflow_uri = os.getenv('MLFLOW_TRACKING_URI')
aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
database_url = os.getenv('DATABASE_URL')

print(f"MLflow URI: {mlflow_uri}")
print(f"AWS Access Key: {aws_access_key}")
print(f"Database URL: {database_url}")

## 4. Usar com MLflow

Configure o MLflow usando variáveis de ambiente:

In [ ]:
import mlflow

# Configurar MLflow com variáveis de ambiente
mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))

print(f"MLflow configurado para: {mlflow.get_tracking_uri()}")

## 5. Usar com credenciais AWS (MinIO)

Configure credenciais AWS para o MLflow acessar o MinIO:

In [ ]:
# Configurar credenciais AWS para MinIO
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['MLFLOW_S3_ENDPOINT_URL'] = os.getenv('MLFLOW_S3_ENDPOINT_URL')

print("✅ Credenciais AWS configuradas para MinIO")

## 6. Usar com valor padrão

Você pode definir um valor padrão caso a variável não exista:

In [ ]:
# Usar valor padrão se a variável não existir
app_name = os.getenv('APP_NAME', 'Default App Name')
debug_mode = os.getenv('DEBUG', 'False')
environment = os.getenv('ENVIRONMENT', 'production')

print(f"App Name: {app_name}")
print(f"Debug Mode: {debug_mode}")
print(f"Environment: {environment}")

## 7. Exemplo Completo - Treinar Modelo com MLflow

Vamos usar todas as configurações para treinar um modelo:

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Carregar dados
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42
)

# Configurar MLflow com variáveis de ambiente
mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
mlflow.set_experiment('iris-classification-with-env')

# Treinar modelo
with mlflow.start_run():
    # Log de parâmetros de ambiente
    mlflow.log_param('environment', os.getenv('ENVIRONMENT', 'unknown'))
    mlflow.log_param('app_name', os.getenv('APP_NAME', 'unknown'))
    
    # Treinar modelo
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Avaliar
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Log de métricas
    mlflow.log_metric('accuracy', accuracy)
    
    # Log de modelo
    mlflow.sklearn.log_model(model, 'model')
    
    print(f"✅ Modelo treinado com acurácia: {accuracy:.2f}")
    print(f"📊 Veja os resultados em: {mlflow.get_tracking_uri()}")

## 8. Boas Práticas

### ✅ Faça:
- Use `.env` para informações sensíveis (senhas, chaves API)
- Adicione `.env` ao `.gitignore`
- Crie um `.env.example` com valores de exemplo
- Use valores padrão com `os.getenv('VAR', 'default')`

### ❌ Não faça:
- Não versione o arquivo `.env` no Git
- Não coloque valores reais no `.env.example`
- Não use `print()` para exibir senhas/chaves secretas
- Não compartilhe seu `.env` publicamente

### 📝 Estrutura recomendada:

```
mlops/
├── .env                    # (gitignored) Valores reais, local
├── .env.example            # (versionado) Valores de exemplo
├── .gitignore              # Inclui .env
└── notebooks/
    └── example_with_env.ipynb
```

## 9. Verificação de Variáveis

Use esta função para verificar se todas as variáveis necessárias estão configuradas:

In [ ]:
def check_env_vars():
    """Verifica se todas as variáveis de ambiente necessárias estão configuradas"""
    required_vars = [
        'MLFLOW_TRACKING_URI',
        'AWS_ACCESS_KEY_ID',
        'AWS_SECRET_ACCESS_KEY',
        'DATABASE_URL'
    ]
    
    missing_vars = []
    for var in required_vars:
        if not os.getenv(var):
            missing_vars.append(var)
    
    if missing_vars:
        print(f"❌ Variáveis faltando: {', '.join(missing_vars)}")
        print("💡 Crie um arquivo .env baseado no .env.example")
        return False
    else:
        print("✅ Todas as variáveis de ambiente estão configuradas!")
        return True

# Verificar variáveis
check_env_vars()

## 🎯 Resumo

1. **Instalar**: `conda install -c conda-forge python-dotenv`
2. **Criar**: Copiar `.env.example` para `.env`
3. **Carregar**: `load_dotenv()`
4. **Usar**: `os.getenv('VARIAVEL')`
5. **Proteger**: Adicionar `.env` ao `.gitignore`

Agora você pode gerenciar configurações sensíveis de forma segura! 🔒